## Imports

In [56]:
import sage.all
from sage.all import GF, ZZ, EllipticCurve, Mod, is_prime, factor
import os
from hashlib import sha256
import random
from operator import xor  

In [57]:

class Edwards25519:
    """
    Implementação da curva Edwards 25519 seguindo estritamente o código original.
    """
    
    def __init__(self):
        # Parâmetros fixos da curva 25519
        self.p = 2**255 - 19
        self.K = GF(self.p)
        self.a = self.K(-1)
        self.d = -self.K(121665)/self.K(121666)
        
        # Calcular constantes da curva
        A = 2*(self.a + self.d)/(self.a - self.d)
        B = 4/(self.a - self.d)
        
        alfa = A/(3*B)
        s = B
        
        a4 = s**(-2) - 3*alfa**2
        a6 = -alfa**3 - a4*alfa
        
        self.constants = {
            'a': self.a,
            'd': self.d,
            'A': A,
            'B': B,
            'alfa': alfa,
            's': s,
            'a4': a4,
            'a6': a6
        }
        
        # Criar curva elíptica equivalente
        self.EC = EllipticCurve(self.K, [a4, a6])
        
        # Definir o subgrupo e ponto gerador
        self.L = ZZ(2**252 + 27742317777372353535851937790883648493)
        self.h = 8  # Cofator
        
        # Coordenadas do ponto gerador em Edwards
        self.Px = self.K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.Py = self.K(46316835694926478169428394003475163141307993866256225615783033603165251855960)
        
        # Converter para a curva elíptica
        self.P = self.ed2ec(self.Px, self.Py)
    
    def is_edwards(self, x, y):
        """
        Verifica se um ponto (x, y) pertence à curva de Edwards.
        """
        a = self.constants['a']
        d = self.constants['d']
        x2 = x**2
        y2 = y**2
        return a*x2 + y2 == 1 + d*x2*y2
    
    def ed2ec(self, x, y):
        """
        Mapeia um ponto da curva Edwards para a curva elíptica equivalente.
        """
        if (x, y) == (0, 1):
            return self.EC(0)  # Ponto no infinito
        
        z = (1 + y)/(1 - y)
        w = z/x
        alfa = self.constants['alfa']
        s = self.constants['s']
        return self.EC(z/s + alfa, w/s)
    
    def ec2ed(self, P):
        """
        Mapeia um ponto da curva elíptica para a curva Edwards.
        """
        if P == self.EC(0):
            return (0, 1)  # Elemento neutro
        
        x, y = P.xy()
        alfa = self.constants['alfa']
        s = self.constants['s']
        u = s*(x - alfa)
        v = s*y
        return (u/v, (u-1)/(u+1))
    
    def order(self):
        """
        Retorna a ordem do subgrupo e o cofator.
        """
        return (self.L, self.h)
    
    def create_point(self, x=None, y=None):
        """
        Cria um ponto na curva Edwards.
        """
        if x is None or y is None:
            return EdPoint(self.Px, self.Py, self)
        
        if self.is_edwards(x, y):
            return EdPoint(x, y, self)
        else:
            raise ValueError("O ponto não pertence à curva Edwards 25519")

In [58]:

class EdPoint:
    """
    Implementação de pontos na curva Edwards, seguindo o código original.
    """
    def __init__(self, x, y, curve):
        self.curve = curve
        self.x = x
        self.y = y
        self.w = x*y
    
    def eq(self, other):
        """
        Verifica se dois pontos são iguais.
        """
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        """
        Cria uma cópia do ponto atual.
        """
        return EdPoint(self.x, self.y, self.curve)
    
    def zero(self):
        """
        Retorna o elemento neutro da curva.
        """
        return EdPoint(0, 1, self.curve)
    
    def sim(self):
        """
        Retorna o simétrico (inverso) do ponto.
        """
        return EdPoint(-self.x, self.y, self.curve)
    
    def soma(self, other):
        """
        Adiciona outro ponto ao ponto atual, alterando o ponto atual.
        """
        a = self.curve.constants['a']
        d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def add(self, other):
        """
        Adiciona outro ponto ao ponto atual, retornando um novo ponto.
        """
        result = self.copy()
        result.soma(other)
        return result
    
    def duplica(self):
        """
        Duplica o ponto atual, alterando o ponto atual.
        """
        a = self.curve.constants['a']
        d = self.curve.constants['d']
        delta = d*(self.w)**2
        self.x, self.y = (2*self.w)/(1+delta), (self.y**2 - a*self.x**2)/(1-delta)
        self.w = self.x*self.y
    
    def double(self):
        """
        Duplica o ponto atual, retornando um novo ponto.
        """
        result = self.copy()
        result.duplica()
        return result
    
    def mult(self, n):
        """
        Multiplicação escalar, seguindo o algoritmo original.
        """
        # Obter representação binária do escalar n módulo L
        m = Mod(n, self.curve.L).lift().digits(2)
        Q = self.copy()
        A = self.zero()
        
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        
        return A

    def create_point(self, x=None, y=None, skip_check=False):
        """
        Cria um ponto na curva Edwards.
        
        Args:
            x: coordenada x (opcional)
            y: coordenada y (opcional)
            skip_check: se True, não verifica se o ponto está na curva
            
        Returns:
            EdPoint: um ponto na curva
        """
        if x is None or y is None:
            return EdPoint(self.Px, self.Py, self)
        
        if skip_check or self.is_edwards(x, y):
            return EdPoint(x, y, self)
        else:
            a = self.constants['a']
            d = self.constants['d']
            x2 = x**2
            y2 = y**2
            left = a*x2 + y2
            right = 1 + d*x2*y2
            
            if abs(left - right) < 1e-10:
                return EdPoint(x, y, self)
            else:
                raise ValueError("O ponto não pertence à curva Edwards 25519")

### Cpa

In [59]:
class EdwardsElGamal:
    """
    Implementação do ElGamal usando a curva Edwards25519
    """
    
    def __init__(self, security_param=128):
        """
        ElGamal em curvas de Edwards 
        """
        self.lambda_security = security_param
        self.curve = Edwards25519()  
        
        """
        Geramos o gerador , calculamos a ordem do grupo e o cofator
        """
        self.G = self.curve.create_point()  
        self.L, self.h = self.curve.order()  
        self.ell = 8  # Número de bits para padding
    
    def keygen(self):
        """
        Gera um par de chaves (privada,pública) para ElGamal a partir do gerador G
        """
        s = random.randint(1, int(self.L) - 1)
        H_point = self.G.mult(s)  #(multiplicação escalar)
        return (H_point, s)
    
    def encode_message(self, message):
        """
        Encodes a fixed-length message (string) into a point on the curve using a standardized Koblitz method.
        
        Steps:
          1. Convert message to integer m.
          2. Verify m fits in (k-1-ell) bits (where k is bit-length of p).
          3. Compute x0 = m << ell.
          4. For i in 0 to 2^ell - 1, let x = x0 + i:
                - Compute f(x) = x^3 + a*x + b mod p.
                - If f(x) is a quadratic residue, let y = sqrt(f(x)) and return the point (x, y).
          5. If no candidate works, raise an error.
        """
        m_int = Integer(int.from_bytes(message.encode('utf-8'), 'big'))
        k_bits = self.curve.p.bit_length()
        if m_int.bit_length() > (k_bits - 1 - self.ell):
            raise ValueError("Message too long to encode in one block.")
        x0 = m_int << self.ell  # Append ell zero bits.
        for i in range(2**self.ell):
            x = x0 + i
            if x >= self.curve.p:
                break
            # Compute f(x) = x^3 + a*x + b mod p.
            f_val = self.curve.K(x**3 + self.curve.constants['a4']*x + self.curve.constants['a6'])
            if f_val.is_square():
                y = f_val.sqrt()
                ec_point = self.curve.EC(x, y)
                ed_x, ed_y = self.curve.ec2ed(ec_point)
                return EdPoint(ed_x, ed_y, self.curve)
        raise ValueError("Non-encodable message: tried 2^ell possibilities.")

    def decode_message(self, point):
        """
        Decodes a point on the Edwards curve back to the original message.
        
        Args:
            point: An EdPoint encoding a message
            
        Returns:
            str: The decoded message
        """
        # Convert Edwards point to Weierstrass point
        ec_point = self.curve.ed2ec(point.x, point.y)
        
        # Extract x-coordinate and remove padding
        x_int = int(ec_point[0])
        m_int = x_int >> self.ell
        
        # Convert back to bytes and then to string
        byte_length = (m_int.bit_length() + 7) // 8
        m_bytes = m_int.to_bytes(byte_length, 'big')
        
        return m_bytes.decode('utf-8')

    def encrypt_message(self, public_key, plaintext):
        """
        Encrypts a message using ElGamal with point encoding
        """
        # Encode the message as a point
        M = self.encode_message(plaintext)
        
        # Generate random value
        omega = random.randint(1, int(self.L) - 1)
        
        # Calculate Gamma = G * omega
        Gamma = self.G.mult(omega)
        
        # Calculate S = public_key * omega
        S = public_key.mult(omega)
        
        # Encrypt: C = M + S
        C = M.add(S)
        
        # Return the ciphertext as (Gamma, C)
        return ((int(Gamma.x), int(Gamma.y)), (int(C.x), int(C.y)))

    def decrypt_message(self, private_key, encrypted_data):
        """
        Decrypts a message using ElGamal with point encoding
        """
        (gamma_x, gamma_y), (c_x, c_y) = encrypted_data
        
        # Recreate the points
        Gamma = self.curve.create_point(self.curve.K(gamma_x), self.curve.K(gamma_y))
        C = self.curve.create_point(self.curve.K(c_x), self.curve.K(c_y))
        
        # Calculate S = Gamma * private_key
        S = Gamma.mult(private_key)
        
        # Get the inverse of S
        S_inv = S.sim()
        
        # Decrypt: M = C - S = C + (-S)
        M = C.add(S_inv)
        
        # Decode the point back to a message
        return self.decode_message(M)

# **Oblivious transfer k-out-of-n**

In [60]:
# Primeiro vou instanciar a classe EdwardsElGamal
ee = EdwardsElGamal()

# Parametros da curva 
q = ee.L     # Ordem do subgrupo da curva
G = ee.G     # Ponto gerador da curva

In [61]:
def assinar(curve_G, priv_key, msg_points):
    """
    Assina uma mensagem representada como uma lista de pontos na curva.
    Retorna a assinatura (R_sign, s).
    """
    k = random.randint(1, q-1)
    R_sign = curve_G.mult(k)
    # Serializa os pontos: concatena as coordenadas dos pontos em msg_points e de R_sign
    msg_bytes = b"".join([str(P.x).encode('utf-8') + str(P.y).encode('utf-8') for P in msg_points])
    msg_bytes += str(R_sign.x).encode('utf-8') + str(R_sign.y).encode('utf-8')
    e = int(hashlib.sha256(msg_bytes).hexdigest(), 16) % q
    s = (k + e * priv_key) % q
    return (R_sign, s)

In [62]:
def verificar_assinatura(curve_G, pub_key, msg_points, assinatura):
    """
    Verifica a assinatura Schnorr sobre uma lista de pontos.
    Retorna True se válida, False caso contrário.
    """
    (R_sign, s) = assinatura
    msg_bytes = b"".join([str(P.x).encode('utf-8') + str(P.y).encode('utf-8') for P in msg_points])
    msg_bytes += str(R_sign.x).encode('utf-8') + str(R_sign.y).encode('utf-8')
    e = int(hashlib.sha256(msg_bytes).hexdigest(), 16) % q
    lhs = curve_G.mult(s)
    rhs = R_sign.add(pub_key.mult(e))
    return (lhs.x == rhs.x and lhs.y == rhs.y)

### **Protocolo oblivious transfer**

In [ ]:
# Parâmetros do protocolo
n = 5   # Número total de mensagens disponíveis
k = 2   # Número de mensagens que o receptor deseja obter
mensagens = [10, 20, 30, 40, 50]  # Exemplo de mensagens numéricas

# Geração de par de chaves do receptor para o esquema ElGamal
sk_R = random.randint(1, q-1)  # Chave privada do receptor
pk_R = G.mult(sk_R)           # Chave pública do receptor

# Geração de par de chaves para assinatura do receptor
sk_R_sign = random.randint(1, q-1)
pk_R_sign = G.mult(sk_R_sign)

# Seleção dos índices escolhidos pelo receptor
indices_escolhidos = [1, 3]  # Exemplo: receptor escolhe os índices 1 e 3

# Construção da requisição OT pelo receptor: para cada índice, gera um ponto R_j
R_list = []
for j in range(n):
    u_j = random.randint(1, q-1)
    R_j = G.mult(u_j)
    if j in indices_escolhidos:
        R_j = R_j.add(pk_R)
    R_list.append(R_j)

# Assinatura da requisição com a chave privada do receptor
assinatura_req = assinar(G, sk_R_sign, R_list)

# O receptor envia (R_list, assinatura_req) para o emissor

# -----------------------------------------------------------------------------
# O EMISSOR PROCESSA A REQUISIÇÃO
# -----------------------------------------------------------------------------

# Geração de chaves para assinatura do emissor
sk_S_sign = random.randint(1, q-1)
pk_S_sign = G.mult(sk_S_sign)

# Emissor verifica a assinatura da requisição
req_valida = verificar_assinatura(G, pk_R_sign, R_list, assinatura_req)
print("Assinatura da requisição válida?", req_valida)

# Emissor cifra cada mensagem m_j usando ElGamal:
# Para cada mensagem, gera um nonce r_j, calcula c1_j = r_j * G, 
# representa a mensagem como M_j = m_j * G e calcula o segredo compartilhado S_j = r_j * R_j,
# obtendo c2_j = M_j + S_j.
cifrases = []
for j, m_j in enumerate(mensagens):
    r_j = random.randint(1, q-1)
    c1_j = G.mult(r_j)
    M_j = G.mult(m_j)
    S_j = R_list[j].mult(r_j)
    c2_j = M_j.add(S_j)
    cifrases.append((c1_j, c2_j))

# Emissor assina a resposta (lista dos pares cifrados)
assinatura_resp = assinar(G, sk_S_sign, [c1 for (c1, c2) in cifrases] + [c2 for (c1, c2) in cifrases])

# Emissor envia (cifrases, assinatura_resp) para o receptor

# -----------------------------------------------------------------------------
# O RECEPTOR PROCESSA A RESPOSTA
# -----------------------------------------------------------------------------

# Receptor verifica a assinatura da resposta do emissor
resp_valida = verificar_assinatura(G, pk_S_sign, [c1 for (c1, c2) in cifrases] + [c2 for (c1, c2) in cifrases], assinatura_resp)
print("Assinatura da resposta válida?", resp_valida)

# Receptor decifra somente as mensagens escolhidas:
# Para cada índice escolhido j, calcula S_j = sk_R * c1_j e recupera M_j_dec = c2_j - S_j.
# Em seguida, realiza uma busca exaustiva para encontrar o escalar m_j tal que M_j_dec = m_j * G.
mensagens_decifradas = {}
for j in indices_escolhidos:
    c1_j, c2_j = cifrases[j]
    S_j = c1_j.mult(sk_R)
    # Calcula o inverso aditivo de S_j: -S_j é equivalente a S_j multiplicado por (q-1)
    M_j_dec = c2_j.add(S_j.mult(q-1))
    decifrado = None
    # Busca exaustiva (assumindo m_j pequeno)
    for candidate in range(0, 100):
        if G.mult(candidate).x == M_j_dec.x and G.mult(candidate).y == M_j_dec.y:
            decifrado = candidate
            break
    mensagens_decifradas[j] = decifrado

# Exibição das mensagens decifradas pelo receptor
print("\nMensagens decifradas pelo receptor:")
for j, msg in mensagens_decifradas.items():
    print("Índice {}: Mensagem = {}".format(j, msg))

# Resumo final
print("\nResumo Final:")
print("Mensagens originais do emissor:", mensagens)
print("Índices escolhidos pelo receptor:", indices_escolhidos)
print("Mensagens decifradas:", [mensagens_decifradas[j] for j in indices_escolhidos])
